In [12]:
!git clone https://gitlab.com/hybit1/massca.git

Cloning into 'massca'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 100 (delta 16), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (100/100), 23.19 MiB | 18.73 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [13]:
import pandas as pd
import numpy as np
import os

In [14]:
def square(data, col='steptime', func=np.mean, alg=['cohda', 'default']):
    a = np.arange(len(alg) ** 2, dtype=float).reshape(len(alg), len(alg))
    for x, i in enumerate(alg):
      for y, j in enumerate(alg):
        df = data[(i, j)]
        a[x, y] = func(data[(i, j)][col])
    return pd.DataFrame(a, index=alg, columns=alg)

In [15]:
base_dir = '/content/massca/examples/cohda/results'
alg = ['cohda', 'default']
data = {}
for i in alg: # --within {i} --between {j}
  for j in alg:
    df = pd.read_csv(os.path.join(base_dir, f"{i}_{j}_results.csv")).rename(columns={
        'MASSCA-0.MosaikAgent-steptime' : 'steptime',
        'GridSim-0.ExternalGrid-0-value' : 'ExternalGrid',
    })
    df['steptime'] = df['steptime'].shift(1)
    df['production'] = df['GridSim-0.Bus-3-value'] + df['GridSim-0.Bus-5-value']
    df['consumption'] = df['GridSim-0.Bus-2-value'] + df['GridSim-0.Bus-4-value']
    df = df.iloc[1::2]
    data[(i, j)] = df

In [16]:
table = []
for i in alg:
  for j in alg:
    df = data[(i, j)]
    table.append({'within' : i,
                  'between' : j,
                  'runtime' : f"{df['steptime'].mean():.2f}",
                  'grid balance' : f"{df['ExternalGrid'].sum():.2f}",
                  'total production' : f"{df['production'].sum():.2f}",
                  'total consumption' : f"{df['consumption'].sum():.2f}"})
pd.DataFrame(table)

,within,between,runtime,grid balance,total production,total consumption
0,cohda,cohda,4.02,1.62,-39.27,40.68
1,cohda,default,2.19,0.05,-40.85,40.68
2,default,cohda,2.06,-4.87,-45.78,40.68
3,default,default,0.03,0.20,-40.68,40.68


In [17]:
square(data)

,cohda,default
cohda,4.020126,2.185506
default,2.056879,0.031254


In [18]:
square(data, 'ExternalGrid', np.sum)

,cohda,default
cohda,1.622061,0.051300
default,-4.869235,0.201138


In [19]:
square(data, 'production', np.sum)

,cohda,default
cohda,-39.270676,-40.850454
default,-45.776142,-40.682692


In [20]:
square(data, 'consumption', np.sum)

,cohda,default
cohda,40.682692,40.682692
default,40.682692,40.682692
